# UE ROLL Blue speaker detection
A model trained to detect Blue UE roll blutooth speaker.
Only 16 images has been used for current training. Performace could be increased using more images.

#Environment Setup

In [12]:
%cd /home/phoenix/Documents/models-master/research

!protoc object_detection/protos/*.proto --python_out=.

%set_env PYTHONPATH=/home/phoenix/Documents/models-master/research:/home/phoenix/Documents/models-master/research/slim

!python object_detection/builders/model_builder_test.py

/home/phoenix/Documents/models-master/research
env: PYTHONPATH=/home/phoenix/Documents/models-master/research:/home/phoenix/Documents/models-master/research/slim
......................
----------------------------------------------------------------------
Ran 22 tests in 0.084s

OK


# Imports

In [2]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import cv2
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

# This is needed to display the images.
%matplotlib inline


# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_FROZEN_GRAPH` to point to a new .pb file.  

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [3]:


# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = '/home/phoenix/Documents/pixel_annot/Custom_Mask/IG/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = '/home/phoenix/Documents/pixel_annot/Custom_Mask/dataset/label.pbtxt'

#Number of classes
NUM_CLASSES = 2

## Load a (frozen) Tensorflow model into memory.

In [4]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [5]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

## Helper code

In [6]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

# Detection

In [7]:
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = '/home/phoenix/Documents/pixel_annot/Custom_Mask/dataset/testImages'
PATH_TO_RESULTS_DIR = '/home/phoenix/Documents/pixel_annot/Custom_Mask/dataset/testImages/result'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR,'video18_frame2862.jpeg'),os.path.join(PATH_TO_TEST_IMAGES_DIR,'video8_frame052.jpeg'),os.path.join(PATH_TO_TEST_IMAGES_DIR,'video19_frame1978.jpeg'),os.path.join(PATH_TO_TEST_IMAGES_DIR,'video19_frame4016.jpeg'),os.path.join(PATH_TO_TEST_IMAGES_DIR,'video14_frame1125.jpeg'), os.path.join(PATH_TO_TEST_IMAGES_DIR,'frame1521.jpeg')]
print(TEST_IMAGE_PATHS)
# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)

['/home/phoenix/Documents/pixel_annot/Custom_Mask/dataset/testImages/video18_frame2862.jpeg', '/home/phoenix/Documents/pixel_annot/Custom_Mask/dataset/testImages/video8_frame052.jpeg', '/home/phoenix/Documents/pixel_annot/Custom_Mask/dataset/testImages/video19_frame1978.jpeg', '/home/phoenix/Documents/pixel_annot/Custom_Mask/dataset/testImages/video19_frame4016.jpeg', '/home/phoenix/Documents/pixel_annot/Custom_Mask/dataset/testImages/video14_frame1125.jpeg', '/home/phoenix/Documents/pixel_annot/Custom_Mask/dataset/testImages/frame1521.jpeg']


In [8]:
def run_inference_for_single_image(image, graph):
    with graph.as_default():
        with tf.Session() as sess:
        # Get handles to input and output tensors
            ops = tf.get_default_graph().get_operations()
            all_tensor_names = {output.name for op in ops for output in op.outputs}
            tensor_dict = {}
            for key in ['num_detections', 'detection_boxes', 'detection_scores','detection_classes', 'detection_masks']:
                tensor_name = key + ':0'
                if tensor_name in all_tensor_names:
                    tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(tensor_name)

            if 'detection_masks' in tensor_dict:
                # The following processing is only for single image
                detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
                detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
                # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
                real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
                detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
                detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
                detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                    detection_masks, detection_boxes, image.shape[0], image.shape[1])
                detection_masks_reframed = tf.cast(
                    tf.greater(detection_masks_reframed, 0.5), tf.uint8)
                # Follow the convention by adding back the batch dimension
                tensor_dict['detection_masks'] = tf.expand_dims(
                    detection_masks_reframed, 0)
            
            
            
            image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

            # Run inference
            output_dict = sess.run(tensor_dict,
                                 feed_dict={image_tensor: np.expand_dims(image, 0)})

            # all outputs are float32 numpy arrays, so convert types as appropriate
            output_dict['num_detections'] = int(output_dict['num_detections'][0])
            output_dict['detection_classes'] = output_dict['detection_classes'][0]
            output_dict['detection_masks'] = output_dict['detection_masks'][0]
        return output_dict

In [9]:
def imfill(inp):
    im_th=inp

    # Copy the thresholded image.
    im_floodfill = im_th.copy()
    cv2.imwrite(PATH_TO_RESULTS_DIR+name,im_floodfill)
    # Mask used to flood filling.
    # Notice the size needs to be 2 pixels than the image.
    h, w = im_th.shape[:2]
    mask = np.zeros((h+2, w+2), np.uint8)

    # Floodfill from point (0, 0)
    cv2.floodFill(im_floodfill, mask, (0,0), 255);
    
    # Invert floodfilled image
    im_floodfill_inv = cv2.bitwise_not(im_floodfill)
    # Combine the two images to get the foreground.
    im_out = im_th | im_floodfill_inv
    return im_out

In [10]:
for image_path in TEST_IMAGE_PATHS:
    image = Image.open(image_path)
    image = image.resize((512, 384), Image.ANTIALIAS)
    # the array based representation of the image will be used later in order to prepare the
    # result image with boxes and labels on it.
    image_np = load_image_into_numpy_array(image)
    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image_np, axis=0)
    # Actual detection.
    output_dict = run_inference_for_single_image(image_np, detection_graph)
    name="/"+image_path.split("/")[-1]  
    mask=output_dict['detection_masks'][0]
    print(mask.shape)
    ret, mask = cv2.threshold(mask, 0, 255, cv2.THRESH_BINARY)
    #kernel = np.ones((5,5), np.uint8)
    #mask = cv2.dilate(mask, kernel, iterations=1)
    #mask = cv2.erode(mask, kernel, iterations=1)
    mask=imfill(mask)
    
    im2, contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    length = len(contours)
    #concatinate poits form all shapes into one array
    cont = np.vstack(contours[i] for i in range(length))

    hull = cv2.convexHull(cont)
    
     
    uni_hull = []
    uni_hull.append(hull)
    
    # draw contours and hull points
    for i in range(len(contours)):
        color = (255, 0, 0) # blue - color for convex hull
        # draw ith convex hull object
        cv2.drawContours(image_np, uni_hull, -1, color, 4, 8)
    
    cv2.imwrite(PATH_TO_RESULTS_DIR+name,image_np)
    
    

(384, 512)


ValueError: not enough values to unpack (expected 3, got 2)